In [7]:
import os, json

import unstructured_client
from unstructured_client.models import operations, shared

client = unstructured_client.UnstructuredClient(
    api_key_auth=os.getenv("UNSTRUCTURED_API_KEY")
)

filename = "docs/qlora.pdf"

req = operations.PartitionRequest(
    partition_parameters=shared.PartitionParameters(
        files=shared.Files(
            content=open(filename, "rb"),
            file_name=filename,
        ),
        strategy=shared.Strategy.VLM,
        vlm_model="gpt-4o",
        vlm_model_provider="openai",
        languages=['eng'],
        split_pdf_page=False,            # If True, splits the PDF file into smaller chunks of pages.
        split_pdf_allow_failed=True,    # If True, the partitioning continues even if some pages fail.
        split_pdf_concurrency_level=15  # Set the number of concurrent request to the maximum value: 15.
    ),
)
try:
    res = client.general.partition(
        request=req
    )
#     element_dicts = [element for element in res.elements]
    
#     # Print the processed data's first element only.
#     print(element_dicts[0])

#     # Write the processed data to a local file.
#     json_elements = json.dumps(element_dicts, indent=2)

#     with open("retule", "w") as file:
#         file.write(json_elements)
except Exception as e:
    print(e)

INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 422 Unprocessable Entity"
ERROR: Server responded with 422 - {"detail":"File does not appear to be a valid PDF"}


{"detail":"File does not appear to be a valid PDF"}


In [ ]:
req.part

AttributeError: 'PartitionParameters' object has no attribute 'content'